In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../Languages.csv')

In [3]:
data.head()

,text,language
0,কাৰাবন্দীৰ পৰা গৃহবন্দী হৈ উভতিল আং ছান ছু কী,Assamese
1,চীন কোৰিয়া ইণ্ডোনেছিয়াৰ পৰা আমদানি কৰা অপটিক...,Assamese
2,গ্ৰেপ্তাৰ হল ইমৰান খানদুৰ্নীতিৰ গোচৰত তিনিবছৰৰ...,Assamese
3,এইবাৰ পাকিস্তানৰ যুৱকৰ সৈতে বিয়াত বহিল দুই সন...,Assamese
4,নিষেধাজ্ঞা নেওচি ভাৰতলৈ নেপালৰ বিলাহীঘোচ লৈ চা...,Assamese


In [4]:
with open('../fasttext_dataset.txt', 'w', encoding='utf-8') as f:
    for index, row in data.iterrows():
        text = row['text']
        language = row['language']
        f.write(f'__label__{language} {text}\n')

In [9]:
!pip install fasttext-wheel scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:0031m225.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 439.0 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 272.3 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 401.1 kB/s eta 0:00:00m eta 0:00:010:00:03


In [6]:
import fasttext

In [7]:
training_data_path = '../fasttext_dataset.txt'

In [10]:
from sklearn.model_selection import train_test_split

In [12]:
X = data['text']
y = data['language']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
train = pd.DataFrame({'text': X_train, 'language': y_train})
test = pd.DataFrame({'text': X_test, 'language': y_test})

In [16]:
with open('../fasttext_train.txt', 'w', encoding='utf-8') as f:
    for index, row in train.iterrows():
        text = row['text']
        language = row['language']
        f.write(f'__label__{language} {text}\n')

In [17]:
with open('../fasttext_test.txt', 'w', encoding='utf-8') as f:
    for index, row in test.iterrows():
        text = row['text']
        language = row['language']
        f.write(f'__label__{language} {text}\n')

In [18]:
model_save_path = '../models/fasttext_finetuned.bin'

In [19]:
model = fasttext.train_supervised(input="../fasttext_train.txt", epoch=100, lr=1.0)

Read 15M words
Number of words:  951567
Number of labels: 10
Progress: 100.0% words/sec/thread: 1212134 lr:  0.000000 avg.loss:  0.001210 ETA:   0h 0m 0s


In [20]:
model.save_model(model_save_path)

In [21]:
loaded_model = fasttext.load_model(model_save_path)

In [22]:
test_data_path = '../fasttext_test.txt'

In [23]:
result = loaded_model.test(test_data_path)

In [24]:
result

(311999, 0.9944967772332604, 0.9944967772332604)

In [25]:
text = "ગુજરાતી ‍(/ɡʊdʒəˈrɑːti/[૬], રોમન લિપિમાં: Gujarātī, ઉચ્ચાર: [ɡudʒəˈɾɑːtiː]) ભારત દેશના ગુજરાત રાજ્યની ઇન્ડો-આર્યન ભાષા છે અને મુખ્યત્વે ગુજરાતી લોકો દ્વારા બોલાય છે. તે બૃહદ ઇન્ડો-યુરોપિયન ભાષા કુટુંબનો ભાગ છે. ગુજરાતીનો ઉદ્ભવ જૂની ગુજરાતી ભાષા (આશરે ઇ.સ. ૧૧૦૦-૧૫૦૦)માંથી થયો છે. તે ગુજરાત રાજ્ય અને દીવ, દમણ અને દાદરા-નગર હવેલી કેન્દ્રશાસિત પ્રદેશોની અધિકૃત ભાષા છે."
loaded_model.predict(text)

(('__label__Gujarati',), array([1.00001001]))